# Update

In [1]:
import pandas as pd

## Database connection

In [2]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [3]:
updates_dimension = pd.read_sql_table("mensajeria_tiponovedad", OLTP_connection)

In [4]:
updates_dimension.head(2)

,id,nombre
0,2,No puedo continuar
1,1,Novedades del servicio


## Transformation

In [5]:
updates_dimension = updates_dimension.rename(
	columns={
		"id": "update_id",
		"nombre": "update_name",
	}
)

In [6]:
updates_dimension.head(2)

,update_id,update_name
0,2,No puedo continuar
1,1,Novedades del servicio


## Load

In [7]:
updates_dimension.to_sql(
	"UPDATES_DIMENSION", OLAP_connection, if_exists="replace", index=False
)

2